# HuggingFaceH4/CodeAlpaca_20K Python split

In [4]:
import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
from datasets import  load_dataset

h4ca20k = load_dataset("HuggingFaceH4/CodeAlpaca_20K")

d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(h4ca20k)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 18019
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2003
    })
})


In [11]:
print(h4ca20k["train"])

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 18019
})


In [14]:
from datasets import Dataset, DatasetDict, concatenate_datasets

In [40]:
from datasets import Features, Value

In [39]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="2200140b76abf3930bc2824cbbaf1310.NeYsf05Dfo7R1z3Y") 

In [23]:
def is_py(prompts):
    py_dataset = Dataset.from_dict({})
    for i in prompts:

        response = client.chat.completions.create(
            model="glm-4", 
    
            messages=[


                {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
                {"role": "user", "content": i["prompt"]},
                
        
            ],
        )

        if(response.choices[0].message.content == "1"):
            py_dataset.add_item(i)

    return py_dataset


In [26]:
prompts  = h4ca20k["test"]

In [56]:

features = Features({
    'prompt': Value('string'),
    'completion': Value('string')
})


py_dataset = Dataset.from_dict({'prompt':[], 'completion':[]},features=features)

In [57]:

cot = 0
for i in prompts:

    response = client.chat.completions.create(
        model="glm-4", 
    
        messages=[


            {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
            {"role": "user", "content": i["prompt"]},
                
        
        ],
    )

    if(response.choices[0].message.content == "1"):
        
        cot = cot+1
        print(cot)

        py_dataset = concatenate_datasets([py_dataset,Dataset.from_dict({'prompt':[i['prompt']],'completion':[i['completion']]})])
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [54]:
py_test = py_dataset

In [55]:
print(py_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 1
})


In [ ]:
def filter_entries(example):
    if is_python_related(example['prompt']):
        return example

filtered_dataset = dataset.map(filter_entries, batched=True)


In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# Convert the filtered dataset to a PyArrow Table
table = pa.Table.from_pandas(filtered_dataset.to_pandas())

# Save the table to a Parquet file
pq.write_table(table, 'filtered_dataset.parquet')
